# Clean Data
## Introduction
To measure the quality of the air we can use Sulfur Monoxide (SO), Nitrogen Dioxide (NO2), Carbon Dioxide (CO2), Ozone (O3) or Total Suspended Particules. The data obtained by OpenAQ is provided by the different sensors around the world. Nevertheless, we will focus on our country during the last year. We consider using 2020 because it is a year that we have a pandemic situation and we would know what happens during the lockdown etc.

## Get started

>Note: this notebook uses python 3 as kernel

This notebook assumes tha data is already downloaded and stored at ``../data/raw``

if not, execute:

``python ../src/data/get_data.py``

## Read Files

We will be using, at first instance, the following python modules:
- ``os`` python built-in package to deal with directories/files
- ``json`` python built-in package to code/encode JSON data
- ``pandas`` to read and process the data
- ``pickle`` to keep stored the read and processed data as a byte stream to save loading times

In [ ]:
import pandas as pd
import os
import json
from pandas_profiling import ProfileReport
from IPython.display import display
import pickle as pkl

# global vars
rel_path="../data/raw/"
clean_path="../data/clean/"
pkl_path="../data/interim/"
rep_path="../reports/"

pkl_parameters = pkl_path + "parameters.pkl"
pkl_countries = pkl_path + "countries.pkl"
pkl_measurements = pkl_path + "measurements.pkl"

os.makedirs(pkl_path, exist_ok=True)
os.makedirs(rep_path, exist_ok=True)

def read_parameters():
    _json = json.load(open(rel_path + 'parameters.json'))
    data = pd.DataFrame(_json["results"]).set_index('id')
    data.to_pickle(pkl_parameters, compression='infer', protcol=-1)
    return data

parameters = pkl.load(open(pkl_parameters, 'rb')) if os.path.exists(pkl_parameters) else read_parameters()
parameters.info()

# Profile report
profile = ProfileReport(parameters, title="Profiling Parameters")
profile.to_widgets()

profile.to_file(rep_path + 'parameters.html')
profile.to_file(rep_path + 'parameters.json')

# Display
parameters = parameters[['displayName','name','preferredUnit']]
display(parameters)

os.makedirs(clean_path, exist_ok=True)
parameters.to_json(clean_path + "parameters.json", orient='records')

In [ ]:
def read_countries():
    _json = json.load(open(rel_path + 'countries.json'))
    data = pd.DataFrame(_json['results'])
    data.to_pickle(pkl_countries, compression='infer', protocol=-1)
    return data

countries = pkl.load(open(pkl_countries, 'rb')) if os.path.exists(pkl_countries) else read_countries()
countries.info()

# Profile report
profile = ProfileReport(countries.loc[:, countries.columns != 'count'], title="Profiling Countries") # cannot insert count, already exists
profile.to_widgets()

profile.to_file(rep_path + 'countries.html')
profile.to_file(rep_path + 'countries.json')

# Display
countries = countries[['cities','code','name','parameters']]
display(countries)

spain = countries[countries['name'] == 'Spain']
display(spain)

print(spain['parameters'].values[0])

countries.to_json(clean_path + "countries.json", orient='records')

In [ ]:
def read_measurements():
    _json = json.load(open(rel_path + 'measurements_day_0.json'))
    data = pd.DataFrame(_json['results'])
    data.to_pickle(pkl_measurements, compression='infer', protocol=-1)
    return data

measurements = pkl.load(open(pkl_measurements, 'rb')) if os.path.exists(pkl_measurements) else read_measurements()
measurements.info()

# Profile report
profile = ProfileReport(measurements, title="Profiling Measurements")
profile.to_widgets()
#measurements.loc['2020-12-31','o3']

profile.to_file(rep_path + 'measurements_day_0.html')
profile.to_file(rep_path + 'measurements_day_0.json')

# Display
measurements = measurements[['average', 'day', 'parameter', 'unit']]
pd.to_datetime(measurements['day'], format='%Y-%m-%d')

measurements.to_json(clean_path + "measurements.json", orient='records')
display(measurements)